In [117]:
from os import walk
import pandas as pd
import numpy as np

cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

f = []
for (dirpath, dirnames, filenames) in walk('../blend/'):
    f.extend(filenames)
    break

ff = []
for (dirpath, dirnames, filenames) in walk('../skolbachev/'):
    for d in dirnames:
        for (dirpath, dirnames2, filenames) in walk('../skolbachev/'+d):
            for qf in filenames:
                ff.append('../skolbachev/'+d+'/'+qf)

train_idx = pd.read_csv('../input/train.csv')['id'].values
train = pd.read_csv('../input/train.csv').sort_values('id').reset_index(drop=True)

In [118]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import minmax_scale
oofs = []
onms = []

train_files = [q for q in f if q.startswith('train')]
for q in train_files:
    nm = q[6:-4]
    nf = pd.read_csv('../blend/'+q).sort_values('id').reset_index(drop=True)
    for c in cols:
        nf[c] = minmax_scale(nf[c])
        #print(nm,c,roc_auc_score(train[c],nf[c]))
    nf.columns = ['id'] + [nm+'_' + q for q in cols]
    print(nm, roc_auc_score(train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']],nf[[nm+'_toxic',nm+'_severe_toxic',nm+'_obscene',nm+'_threat',nm+'_insult',nm+'_identity_hate']]))
    onms.append(nm)
    oofs.append(nf)
    
sk_train = [q for q in ff if not q.endswith('test_X_pred.npy')]
suf = 'sk'
i = 0
for q in sk_train:
    nf = pd.DataFrame(np.load(q))
    nm = suf+str(i)
    nf.columns = [nm+'_'+q for q in cols]
    nf['id'] = train_idx
    nf = nf.sort_values('id').reset_index(drop=True)
    for c in cols:
        nf[nm+'_'+c] = minmax_scale(nf[nm+'_'+c])
    print(nm, roc_auc_score(train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']],nf[[nm+'_toxic',nm+'_severe_toxic',nm+'_obscene',nm+'_threat',nm+'_insult',nm+'_identity_hate']]))
    onms.append(nm)
    oofs.append(nf)
    i = i + 1

_att 0.9887701275576107
2ycnn 0.9873606598169475
4gru 0.9876478836087786
5cnn 0.9852476095999143
att 0.9880287618175224
grucnn 0.9895349799724892
grucnn_fl 0.9880106712925772
grucnn_fl3 0.9897685069751564
grucnn_fl4 0.9899923857692398
nbsvm 0.9820123460744199
ycnn 0.9847277643406013
sk0 0.9910875692487013
sk1 0.9908130451768224
sk2 0.9899530158503688
sk3 0.9911913850256058
sk4 0.9905625848095353
sk5 0.9902882130480376
sk6 0.990813388694949
sk7 0.9904216084142267
sk8 0.9911659724246551
sk9 0.9907451200741724


In [119]:
train = pd.read_csv('../input/train.csv').sort_values('id').reset_index(drop=True)
for o in oofs:
    train = train.merge(o, on='id', how='left')

In [120]:
onms

['_att',
 '2ycnn',
 '4gru',
 '5cnn',
 'att',
 'grucnn',
 'grucnn_fl',
 'grucnn_fl3',
 'grucnn_fl4',
 'nbsvm',
 'ycnn',
 'sk0',
 'sk1',
 'sk2',
 'sk3',
 'sk4',
 'sk5',
 'sk6',
 'sk7',
 'sk8',
 'sk9']

In [96]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', '_att_toxic', '_att_severe_toxic',
       ...
       'sk7_obscene', 'sk7_threat', 'sk7_insult', 'sk7_identity_hate',
       'sk8_toxic', 'sk8_severe_toxic', 'sk8_obscene', 'sk8_threat',
       'sk8_insult', 'sk8_identity_hate'],
      dtype='object', length=128)

In [121]:
orig_cols = ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [122]:
from sklearn.linear_model import Ridge

In [99]:
from sklearn.model_selection import cross_val_score
for c in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    y = train[c]
    q = train.drop(orig_cols, axis=1)[['grucnn_fl4_toxic',
       'grucnn_fl4_severe_toxic', 'grucnn_fl4_obscene', 'grucnn_fl4_threat',
       'grucnn_fl4_insult', 'grucnn_fl4_identity_hate']]
    c = cross_val_score(Ridge(), q, y, scoring='roc_auc', cv=10, n_jobs=4)
    print(c,np.mean(c))

[0.98472914 0.98562069 0.98650374 0.98598169 0.98558993 0.98480296
 0.98168425 0.98371927 0.98639106 0.98581688] 0.985083962437276
[0.99074378 0.98775953 0.98442865 0.99211569 0.98704678 0.98335224
 0.97720848 0.98032745 0.98054624 0.98509982] 0.9848628648050239
[0.98994167 0.99040779 0.98843374 0.9901125  0.99329596 0.99098916
 0.99116923 0.99404594 0.99215182 0.99072418] 0.9911271991238454
[0.99236474 0.99477086 0.99541594 0.98515622 0.99722962 0.99424732
 0.98207517 0.97125441 0.99600022 0.99539668] 0.9903911175832661
[0.98520965 0.98264065 0.98615862 0.98419373 0.98673295 0.98316835
 0.98073277 0.98369183 0.98875661 0.98399784] 0.9845283007320557
[0.9758174  0.98416528 0.98560318 0.98232427 0.99108667 0.97647954
 0.9618444  0.98038886 0.97933769 0.97923957] 0.9796286851281062


In [100]:
from sklearn.model_selection import cross_val_score
for c in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    y = train[c]
    q = train.drop(orig_cols, axis=1)
    c = cross_val_score(Ridge(alpha=0.01), q, y, scoring='roc_auc', cv=10, n_jobs=4)
    print(c,np.mean(c))

[0.9886     0.98895616 0.98862214 0.98938489 0.99027366 0.98863888
 0.9868265  0.98795015 0.98823728 0.98988634] 0.9887375998314265
[0.98566743 0.98167486 0.98539811 0.98703947 0.98757604 0.97436425
 0.99038048 0.97521192 0.99317423 0.9844962 ] 0.9844982993019343
[0.99415615 0.99573892 0.99462091 0.99579508 0.99327641 0.99249492
 0.99347025 0.99249526 0.99475028 0.99323542] 0.9940033599040058
[0.98782104 0.99594518 0.99738479 0.93685547 0.99828609 0.99450476
 0.96383614 0.97058768 0.99669711 0.99632123] 0.9838239498308846
[0.98378575 0.98822147 0.99060971 0.99061278 0.98980544 0.98494558
 0.98599986 0.99041442 0.99162943 0.98881515] 0.9884839581916658
[0.98032598 0.98504133 0.99433088 0.97300961 0.98119227 0.98668671
 0.95962165 0.97685965 0.97713227 0.98471917] 0.9798919515308547


In [123]:
scores = {}
for c in cols:
    y = train[c]
    q = train.drop(orig_cols, axis=1)
    scores[c] = []
    for n in onms:
        w = roc_auc_score(y,train[n+'_'+c])
        scores[c].append(w)
        #print(n,w)

In [124]:
scores

{'identity_hate': [0.986755867062143,
  0.9853225493122388,
  0.987369281780307,
  0.9842576223916824,
  0.986799649613589,
  0.987598454491009,
  0.9851903736616555,
  0.9887543124991929,
  0.9883244024488349,
  0.9743026325375614,
  0.9830583823302361,
  0.9893201196832573,
  0.988678816341568,
  0.9877394388516448,
  0.990484253153912,
  0.988861659512374,
  0.9881943935384252,
  0.9893613642462132,
  0.9888579065293939,
  0.9895454854112237,
  0.9886383165252346],
 'insult': [0.9874042312040496,
  0.986756542175179,
  0.9863483058397539,
  0.9846323715215898,
  0.9871613275758357,
  0.988039410332855,
  0.9853919458227126,
  0.9882840685105397,
  0.9882289928971904,
  0.9814768203661656,
  0.984986247788525,
  0.9897227191530162,
  0.9895313774362904,
  0.9888093089838679,
  0.9896438924717081,
  0.9891784719670589,
  0.9890702723785026,
  0.9893435448185054,
  0.9890715323232812,
  0.9897561543465176,
  0.9894528281855326],
 'obscene': [0.9927528671719308,
  0.9925781845291518,
  

In [125]:
p = []
for c in cols:
    ws = scores[c]
    y = train[c]
    pred = 0
    i = 0
    for n in onms:
        pred += ((ws[i]-np.min(ws))/(np.max(ws)-np.min(ws))+0.1)*train[n+'_'+c]
    p.append(roc_auc_score(y,pred))
    print(c,roc_auc_score(y,pred))
np.mean(p)

toxic 0.9887227450268732
severe_toxic 0.9923648495156521
obscene 0.9952125067058077
threat 0.9952909178381941
insult 0.9902860642731115
identity_hate 0.9909840568873022


0.9921435233744901

In [126]:
preds = []

train_files = [q for q in f if q.startswith('test')]
for q in train_files:
    nf = pd.read_csv('../blend/'+q)
    for c in cols:
        nf[c] = minmax_scale(nf[c])
    preds.append(nf)
    
sk_train = [q for q in ff if q.endswith('test_X_pred.npy')]
suf = 'sk'
i = 0
for q in sk_train:
    nf = pd.DataFrame(np.mean(np.load(q),axis=0))
    nf.columns = cols
    for c in cols:
        nf[c] = minmax_scale(nf[c])
    preds.append(nf)
    i = i + 1

In [127]:
len(preds)

21

In [128]:
len(ws)

21

In [129]:
sub = pd.read_csv('../input/sample_submission.csv')
for c in cols:
    sub[c] = 0
    ws = scores[c]
    for j in range(len(ws)):
        sub[c] += ((ws[j]-np.min(ws))/(np.max(ws)-np.min(ws))+0.1) * preds[j][c]
    sub[c] = minmax_scale(sub[c])
sub.head(n=3)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.987099,0.433757,0.966528,0.143219,0.936795,0.545967
1,0000247867823ef7,0.004816,0.000764,0.002298,0.000717,0.001950,0.001048
2,00013b17ad220c46,0.005392,0.001375,0.003499,0.001403,0.002486,0.001882


In [130]:
sub.to_csv('weighted_blend_21models.csv', index=False)